In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [2]:
#path = r'C:\Users\Admin\Desktop\program\datasets\cifar-10-batches-py\data_batch_1'

def _unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
def cifar_parser(file_path, batch_size):
    ''' Takes the path to a pickled file as input and returns a numpy
        array. 'iter()' can be used to create an iterator and 'next()'
        can be used to generate a batch of images everytime.
        
        Args:
            file_path : The full path of the file as a raw string
            batch_size : The number of images to generate at each batch
    '''
    
    
    d = _unpickle(file_path)
    num_images, img_size = d[b'data'].shape[0], d[b'data'].shape[1]
    
    # batch_data is just the entire data of the pickle file as a numpy array
    # which is reshaped in such a way that it is easy to create an iterator
    # of the array and generate a batch everytime 'next(batch_size)' is run
    batch_data = d[b'data'][:].reshape(num_images//batch_size,batch_size,3,img_size//3).reshape(num_images//batch_size,batch_size,3,int((img_size//3)**0.5),int((img_size//3)**0.5)).transpose((0,1,3,4,2))
    return batch_data

In [4]:
def cifar_iterator(folder_path, batch_size):
    ''' function that takes the folder path
        where all the cifar_10 files are present. 'iter()' can be used 
        to create an iterator and 'next()' can be used to generate a
        batch of images everytime
        
        Args:
            folder_path : The full path of the folder where all the cifar_10
                            files are present. raw string input
            batch_size : The number of images to generate at each batch
    '''
    
    # make an os.listdir() to get the names of all the files
    # get their paths by joining the folder path and the file name
    
    # The function will yield the 'batch_data' for each file and the 'batch_data'
    # can then be iterated "iter(batch_data)" during training to get each batch
    
    # so we are creating a generator from a generator
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        yield cifar_parser(file_path, batch_size)

In [5]:
#file_iterator = cifar_iterator(r'C:\Users\Admin\Desktop\program\datasets\cifar-10-batches-py', 50)

#num_files = len(os.listdir(r'C:\Users\Admin\Desktop\program\datasets\cifar-10-batches-py'))

def batch_generator(folder_path, batch_size):
    file_iterator = cifar_iterator(folder_path, batch_size)
    num_files = len(os.listdir(folder_path))
    
    for _ in range(num_files):
        file_data = next(file_iterator) # load a file and parse it into (200,50,32,32,3) array
        batch_iterator = iter(file_data)
        
        for _ in range(file_data.shape[0]):
            batch_data = next(batch_iterator) # outputs a batch of size (50,32,32,3) from the file of size (200,50,32,32,3)
            yield batch_data

In [6]:
def helper_conv2d_transpose(inputs, filters, strides = (2,2), padding = 'same'):
    return tf.layers.conv2d_transpose(inputs = inputs,
                                        filters = filters, 
                                        kernel_size = 4, 
                                        strides = strides, 
                                        padding = padding, 
                                        use_bias = False, 
                                        kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d())

In [7]:
def helper_conv2d(inputs, filters, strides = (2,2), padding = 'same'):
    return tf.layers.conv2d(inputs = inputs,
                                        filters = filters, 
                                        kernel_size = 4, 
                                        strides = strides, 
                                        padding = padding, 
                                        use_bias = False, 
                                        kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d())

In [8]:
def helper_loss(labels_in, logits_in):
    return tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_in, logits=logits_in)

In [9]:
def generator(g_image, reuse=None):
    with tf.variable_scope('g', reuse=reuse):
        
        # g_image will have the random noise with (1x1x100) dimensions
        gnet = helper_conv2d_transpose(g_image, 512, (1,1), 'valid')
        gnet = tf.layers.batch_normalization(inputs = gnet)
        gnet = tf.nn.relu(gnet)

        gnet = helper_conv2d_transpose(gnet, 256)
        gnet = tf.layers.batch_normalization(inputs = gnet)
        gnet = tf.nn.relu(gnet)
                
        gnet = helper_conv2d_transpose(gnet, 128, (1,1))
        gnet = tf.layers.batch_normalization(inputs = gnet)
        gnet = tf.nn.relu(gnet)
              
        gnet = helper_conv2d_transpose(gnet, 64)
        gnet = tf.layers.batch_normalization(inputs = gnet)
        gnet = tf.nn.relu(gnet)
                      
        gnet = helper_conv2d_transpose(gnet, 3)
        
        gnet = tf.nn.tanh(gnet)
        
        return gnet

In [10]:
def discriminator(x_image, reuse=None):
    with tf.variable_scope('d', reuse=reuse):
        
        # x_image will take true images and generator images alternatively
        # same weights will be used(reuse=True) to train both fake and real images
        dnet = helper_conv2d(x_image, 64)
        dnet = tf.nn.leaky_relu(dnet)                        
    
        dnet = helper_conv2d(dnet, 128)
        dnet = tf.layers.batch_normalization(inputs = dnet)
        dnet = tf.nn.leaky_relu(dnet)

        dnet = helper_conv2d(dnet, 256)
        dnet = tf.layers.batch_normalization(inputs = dnet)
        dnet = tf.nn.leaky_relu(dnet)
             
        dnet = helper_conv2d(dnet, 512)
        dnet = tf.layers.batch_normalization(inputs = dnet)
        dnet = tf.nn.leaky_relu(dnet)    
        
        #dnet = helper_conv2d(dnet, 1, (1,1), 'valid')
        
        dnet = tf.layers.flatten(dnet)
        
        dnet = tf.layers.dense(dnet, 1024, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
        dnet = tf.layers.dropout(dnet, 0.5)
        
        dnet = tf.layers.dense(dnet, 128, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
        dnet = tf.layers.dropout(dnet, 0.5)
        
        dnet = tf.layers.dense(dnet, 1, kernel_initializer=tf.contrib.layers.xavier_initializer())
        out = tf.nn.sigmoid(dnet)
           
        return out, dnet

In [11]:
# if fetching data from google drive then change the address accordingly

folder_path = r'C:\Users\Admin\Desktop\program\datasets\cifar-10-batches-py'

In [14]:
# placeholder for the input images(discriminator) and noise(generator)

input_image = tf.placeholder(tf.float32, [None, 32,32,3]) # feed input images
gen_image = tf.placeholder(tf.float32, [None, 1,1,100]) # feed random noise

In [15]:
# create variables for generator and discriminator

gen = generator(gen_image)
dis_output_real, dis_logits_real = discriminator(input_image)
dis_output_fake, dis_logits_fake = discriminator(gen, reuse=True)

In [16]:
# get all the trainable variables to apply regularization

vars_g = [var for var in tf.trainable_variables() if var.name.startswith('g')]
vars_d = [var for var in tf.trainable_variables() if var.name.startswith('d')]

reg_g = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_g)
reg_d = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_d)

In [17]:
dis_loss_real = helper_loss(tf.ones_like(dis_logits_real), dis_logits_real)
dis_loss_fake = helper_loss(tf.zeros_like(dis_logits_fake), dis_logits_fake)

dis_loss = tf.reduce_mean((dis_loss_real + dis_loss_fake)*0.5)
gen_loss = tf.reduce_mean(helper_loss(tf.ones_like(dis_logits_fake), dis_logits_fake))

In [18]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [19]:
with tf.control_dependencies(update_ops):
    optimizer_dis = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(dis_loss + reg_d, var_list = vars_d)
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(dis_loss + reg_d, var_list = vars_d)

In [20]:
#                                                  CAUTION !!!
#                                         only run on cloud platform !!!

'''
with tf.Session() as sess:
    
    batch_size = 50
    
    num_epochs = 25
    num_batches = 200*6
    
    # input for the generator
    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)
    #batch_iter = batch_generator(folder_path, batch_size)
    
    for epoch in range(num_epochs):
        
        # refresh the iterator after each epoch
        batch_iter = batch_generator(folder_path, batch_size)
        new_batch_flag = 1
        batch_counter = 0 # counts number of new batches used
        
            #for i in range(num_batches):
            while batch_counter < num_batches:
                train_d = True
                train_g = True
                
                
                # will be zero if the previous batch was not used to train the discriminator
                # so that the next batch will not be generated (the old batch will be reused)
                # problem : the loop will run 1200 times but 1200 new batches wont be used
                # possible solution : create a counter(for num of new batches used) 
                #     and use while loop inplace of the inner for loop
                
                if new_batch_flag == 1: 
                    batch = next(batch_iter) # generate the next batch out of 200*6 batches
                    batch_counter += 1
                    
                d_real_ls, d_fake_ls, g_ls, d_ls = sess.run([dis_loss_real, dis_loss_fake, gen_loss, dis_loss], 
                                                            feed_dict={input_image: batch, gen_image: n})
                
                d_real_ls = np.mean(d_real_ls)
                d_fake_ls = np.mean(d_fake_ls)
                g_ls = g_ls
                d_ls = d_ls
                
                
                # the code below will create a problem
                # because the next batch will be generated but the 
                # discriminator will not be trained if train_d = False
                # so we are losing some of the training data
                
                # possible solution : generate a batch only "if train_d = True"
                # or remove the constraints on training
                # or create a flag as a feed back for whether or not the batch was used in the previous iteration
                
                if g_ls * 1.5 < d_ls:
                    train_g = False
                    pass
                if d_ls * 2 < g_ls:
                    train_d = False
                    new_batch_flag = 0
                    pass
                if train_d:
                    sess.run(optimizer_d, feed_dict={gen_image: n, input_image: batch})
                    new_batch_flag = 1


                if train_g:
                    sess.run(optimizer_g, feed_dict={gen_image: n})
                

'''

'\nwith tf.Session() as sess:\n    \n    batch_size = 50\n    \n    num_epochs = 25\n    num_batches = 200*6\n    \n    # input for the generator\n    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)\n    #batch_iter = batch_generator(folder_path, batch_size)\n    \n    for epoch in range(num_epochs):\n        \n        # refresh the iterator after each epoch\n        batch_iter = batch_generator(folder_path, batch_size)\n        new_batch_flag = 1\n        batch_counter = 0 # counts number of new batches used\n        \n            #for i in range(num_batches):\n            while batch_counter < num_batches:\n                train_d = True\n                train_g = True\n                \n                \n                # will be zero if the previous batch was not used to train the discriminator\n                # so that the next batch will not be generated (the old batch will be reused)\n                # problem : the loop will run 1200 times but 1200 ne

In [21]:
def generate_samples():
    
    # launch a new session and input some random noise into the generator to get an image
    
    pass